<a href="https://colab.research.google.com/github/stanywandanda-lab/ML_BAC4_UMK/blob/main/Travail%20memoire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLAGIAT MEMOIRES

IMPORTATION DES BIBLIOTHEQUE

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from google.colab import files

Nettoyage

In [ ]:

try:
    df = pd.read_excel('Plagiat.xlsx', sheet_name='Feuil1')
except FileNotFoundError:
    print("Erreur: Le fichier 'votre_fichier.xlsx' n'a pas été trouvé.")

    exit()

# Afficher les premières lignes pour vérifier le chargement
print("Aperçu des données chargées:")
print(df.head())

Aperçu des données chargées:
   N°                                      texte_memoire  \
0   1  Conception d’une plateforme \nweb de dépôt de ...   
1   2  L’impact des réseaux sociaux sur la réussite s...   
2   3    Développement d’une application mobile de suivi   
3   4  Analyse de données pour la prédiction des perf...   
4   5  Sécurité et cryptographie dans le cloud computing   

                                        texte_source score_similarite  \
0  Développement d’une plateforme web avec détection               85   
1      Influence des réseaux sociaux sur la réussite               70   
2   Création d’une application mobile pour étudiants               65   
3   Étude des performances des étudiants via données               55   
4     Protection des données dans le cloud computing               80   

  longueur_texte  
0              8  
1              7  
2              7  
3              9  
4              6  


In [ ]:
# Remplacez 'Ancien Nom Colonne 1' et 'Ancien Nom Colonne 3' par les vrais noms de vos colonnes
df = df.rename(columns={
    'Ancien Nom Colonne 1': 'texte_memoire',
    'Ancien Nom Colonne 3': 'texte_source',
    'Ancien Nom Colonne Cible': 'plagiat'
})

In [ ]:
df = pd.read_excel('Plagiat.xlsx', sheet_name='Feuil1')

Parametres de chargement

In [ ]:

# IMPORTS DES LIBRAIRIES

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix


# Paramètres de chargement
NOM_FICHIER = 'Plagiat.xlsx'
NOM_FEUILLE = 'Feuil1'

# CORRECTION DES NOMS DE COLONNES (PAS D'ACCENT)
COL_MEMOIRE = 'texte_memoire'
COL_SOURCE  = 'texte_source'
# NOM DE LA COLONNE CIBLE (0 ou 1) - DOIT ÊTRE CONFIRMÉ
COL_CIBLE   = 'plagiat'

try:
    df = pd.read_excel(NOM_FICHIER, sheet_name=NOM_FEUILLE)
    print(f"Fichier '{NOM_FICHIER}' chargé avec succès. {len(df)} lignes trouvées.")
except FileNotFoundError:
    print(f" Erreur: Le fichier '{NOM_FICHIER}' n'a pas été trouvé. Veuillez vérifier l'emplacement.")
    exit()


# Ces lignes fonctionnent maintenant avec les noms de colonnes corrigés
df[COL_MEMOIRE] = df[COL_MEMOIRE].fillna('')
df[COL_SOURCE] = df[COL_SOURCE].fillna('')

# Le code va s'arrêter ici car nous ne sommes pas sûrs du nom de la colonne cible
if COL_CIBLE not in df.columns:
    print(f"\n ERREUR CRITIQUE : La colonne cible '{COL_CIBLE}' est manquante.")
    print("Veuillez confirmer le nom exact de la colonne qui contient les étiquettes binaires (0 ou 1) pour le plagiat.")
    exit() # Arrête le script pour que vous puissiez corriger

Fichier 'Plagiat.xlsx' chargé avec succès. 1083 lignes trouvées.

 ERREUR CRITIQUE : La colonne cible 'plagiat' est manquante.
Veuillez confirmer le nom exact de la colonne qui contient les étiquettes binaires (0 ou 1) pour le plagiat.


In [ ]:

NOM_FICHIER = 'Plagiat.xlsx'
NOM_FEUILLE = 'Feuil1'
COL_MEMOIRE = 'texte_memoire'
COL_SOURCE  = 'texte_source'

COL_CIBLE   = 'NOM_DE_LA_COLONNE_CIBLE_0_ou_1'

In [ ]:


try:

    df[COL_MEMOIRE] = df[COL_MEMOIRE].astype(str)
    df[COL_SOURCE] = df[COL_SOURCE].astype(str)

    df[COL_MEMOIRE] = df[COL_MEMOIRE].replace('nan', '')
    df[COL_SOURCE] = df[COL_SOURCE].replace('nan', '')


    df[COL_MEMOIRE] = df[COL_MEMOIRE].fillna('')
    df[COL_SOURCE] = df[COL_SOURCE].fillna('')

    print("Nettoyage des colonnes de texte complété (conversion en string réussie).")

except KeyError as e:
    print(f"Erreur critique : La colonne {e} n'a pas été trouvée. Vérifiez à nouveau les variables COL_MEMOIRE/COL_SOURCE.")
    exit()
except Exception as e:
    print(f" Erreur inattendue durant le nettoyage : {e}")
    exit()

Nettoyage des colonnes de texte complété (conversion en string réussie).


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

# CORRECTION DU PARAMÈTRE 'stop_words'

import nltk
from nltk.corpus import stopwords

# Téléchargement de la liste des stop words
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')

# Définition de la liste française
FRENCH_STOP_WORDS = stopwords.words('french')

#  Ingénierie des Caractéristiques (Mise à jour du vectorizer)


print("--- Création de la Caractéristique de Similarité (X) ---")

# 1.1 & 1.2. Vectorisation TF-IDF (utilisation de la liste FRENCH_STOP_WORDS)
corpus = df[COL_MEMOIRE].tolist() + df[COL_SOURCE].tolist()
vectorizer = TfidfVectorizer(
    lowercase=True,
    # 🚨 CORRECTION : Utilisation de la liste définie par NLTK
    stop_words=FRENCH_STOP_WORDS,
)
vectorizer.fit(corpus) # Apprend le vocabulaire

# ... Le reste du code de l'Étape 1 peut suivre...
# (M_tfidf, S_tfidf, calcul Cosinus, etc.)

--- Création de la Caractéristique de Similarité (X) ---


TfidfVectorizer(stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...])

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:

#  TÉLÉCHARGEMENT DES DONNÉES NLTK


try:
    nltk.data.find('corpora/stopwords')
except:
    nltk.download('stopwords')

# Définition de la liste des mots vides français (celle que TfidfVectorizer utilisera)
FRENCH_STOP_WORDS = stopwords.words('french')

In [ ]:
# Code de l'Étape 1
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words=FRENCH_STOP_WORDS # <-- C'est le remplacement correct !
)
vectorizer.fit(corpus)

TfidfVectorizer(stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...])

In [ ]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix

import nltk
from nltk.corpus import stopwords

# Téléchargement et définition des stop words
try:
    # Tente de trouver les stop words
    nltk.data.find('corpora/stopwords')
except:
    # Les télécharge si non trouvés
    nltk.download('stopwords')

FRENCH_STOP_WORDS = stopwords.words('french')

print("---  Création de la Caractéristique de Similarité (X) ---")

# Vectorisation TF-IDF
corpus = df[COL_MEMOIRE].tolist() + df[COL_SOURCE].tolist()
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words=FRENCH_STOP_WORDS # CORRECTION DÉFINITIVE
)
vectorizer.fit(corpus)

#  Transformation
M_tfidf = vectorizer.transform(df[COL_MEMOIRE])
S_tfidf = vectorizer.transform(df[COL_SOURCE])

#  Calcul Cosinus
scores_similarite = []
for i in range(M_tfidf.shape[0]):
    sim = cosine_similarity(M_tfidf[i], S_tfidf[i])[0][0]
    scores_similarite.append(sim)

# Création de X
df['X_similarite'] = scores_similarite
COL_SCORE_SIMILARITE = 'X_similarite'

print(f"réussie : Colonne '{COL_SCORE_SIMILARITE}' (X) créée.")
print(df[[COL_MEMOIRE, COL_SOURCE, COL_SCORE_SIMILARITE]].head())

---  Création de la Caractéristique de Similarité (X) ---
réussie : Colonne 'X_similarite' (X) créée.
                                       texte_memoire  \
0  Conception d’une plateforme \nweb de dépôt de ...   
1  L’impact des réseaux sociaux sur la réussite s...   
2    Développement d’une application mobile de suivi   
3  Analyse de données pour la prédiction des perf...   
4  Sécurité et cryptographie dans le cloud computing   

                                        texte_source  X_similarite  
0  Développement d’une plateforme web avec détection      0.493844  
1      Influence des réseaux sociaux sur la réussite      0.698948  
2   Création d’une application mobile pour étudiants      0.497858  
3   Étude des performances des étudiants via données      0.363322  
4     Protection des données dans le cloud computing      0.576221  


In [ ]:
#  Préparation et Entraînement du Modèle

print("\n---   : Préparation et Entraînement du Modèle ---")

COL_CIBLE = 'plagiat'
COL_SCORE_SIMILARITE = 'X_similarite' # Colonne créée à l'Étape 1

if COL_CIBLE not in df.columns:
    COL_CIBLE = 'y_plagiat_cree' # Nouvelle colonne créée
    SEUIL_PLAGIAT = 0.70 # Seuil arbitraire de 70% de similarité

    # Création de la cible (y) : 1 si le score est >= 0.70, 0 sinon
    df[COL_CIBLE] = (df[COL_SCORE_SIMILARITE] >= SEUIL_PLAGIAT).astype(int)
    print(f"  Cible (y) créée par seuillage (seuil >= {SEUIL_PLAGIAT}).")
else:
    print(f"Utilisation de la colonne cible existante : {COL_CIBLE}")

# Définition des variables finales pour le ML
X = df[[COL_SCORE_SIMILARITE]] # Notre caractéristique d'entrée
y = df[COL_CIBLE]              # Notre étiquette à prédire

# 2. SÉPARATION TRAIN/TEST

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, # 20% des données réservées au test
    random_state=42,
    stratify=y # Maintient l'équilibre des classes (0 et 1)
)

print(f"Taille de l'ensemble d'entraînement : {len(X_train)} échantillons")
print(f"Taille de l'ensemble de test : {len(X_test)} échantillons")

# ENTRAÎNEMENT DU MODÈLE
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print(" réussie : Modèle Random Forest entraîné sur la similarité (X).")


---   : Préparation et Entraînement du Modèle ---
  Cible (y) créée par seuillage (seuil >= 0.7).
Taille de l'ensemble d'entraînement : 866 échantillons
Taille de l'ensemble de test : 217 échantillons
 réussie : Modèle Random Forest entraîné sur la similarité (X).


In [ ]:
#   Évaluation et Rapport de Performance

print("\n---   : Évaluation et Rapport de Performance ")

#  PRÉDICTION
y_pred = model.predict(X_test)

# ÉVALUATION

print("\n  RAPPORT DE CLASSIFICATION FINAL ")
print("Analyse des performances avec un Seuil de Plagiat de 0.60 :")
print(classification_report(y_test, y_pred))

# Afficher les métriques clés de manière plus lisible
f1_plagiat = f1_score(y_test, y_pred, pos_label=1)
print(f"\nScore F1 pour la classe 1 (Plagiat) : {f1_plagiat:.4f}")

print("\nMatrice de Confusion :")
print(confusion_matrix(y_test, y_pred))

print(" ÉTAPE 3 réussie : L'évaluation du modèle est terminée.")


---   : Évaluation et Rapport de Performance 

  RAPPORT DE CLASSIFICATION FINAL 
Analyse des performances avec un Seuil de Plagiat de 0.60 :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       123
           1       1.00      1.00      1.00        94

    accuracy                           1.00       217
   macro avg       1.00      1.00      1.00       217
weighted avg       1.00      1.00      1.00       217


Score F1 pour la classe 1 (Plagiat) : 1.0000

Matrice de Confusion :
[[123   0]
 [  0  94]]
 ÉTAPE 3 réussie : L'évaluation du modèle est terminée.


In [ ]:

#  FONCTION DE PRÉDICTION DE PLAGIAT



def predire_plagiat(texte_memoire_nouveau, texte_source_nouveau, seuil_plagiat=0.60):


    # Convertir en liste pour la transformation
    nouveaux_textes_memoire = [texte_memoire_nouveau]
    nouveaux_textes_source = [texte_source_nouveau]

    # 1. Transformation en vecteurs TF-IDF
    # IMPORTANT : On utilise le vectorizer déjà entraîné (ne pas refaire vectorizer.fit)
    M_tfidf_new = vectorizer.transform(nouveaux_textes_memoire)
    S_tfidf_new = vectorizer.transform(nouveaux_textes_source)

    # 2. Calcul de la Similarité Cosinus
    score = cosine_similarity(M_tfidf_new, S_tfidf_new)[0][0]

    # 3. Décision basée sur le seuil
    if score >= seuil_plagiat:
        decision = "PLAGIAT DÉTECTÉ (1)"
    else:
        decision = "NON-PLAGIAT (0)"

    return decision, score


# EXEMPLE D'UTILISATION


#  NON-PLAGIAT
texte_A = "L'intelligence artificielle transforme l'éducation et la recherche universitaire."
texte_B = "Les voitures autonomes révolutionnent les transports dans les grandes métropoles."

decision_1, score_1 = predire_plagiat(texte_A, texte_B)
print(f"--- CAS 1 : Non-Plagiat ---")
print(f"Score de Similarité: {score_1:.4f} (Seuil: 0.60)")
print(f"Décision: {decision_1}")
print("-" * 30)


# PLAGIAT DÉTECTÉ (haute similarité)
texte_C = "L'utilisation de modèles linguistiques avancés permet d'améliorer l'automatisation des tâches répétitives dans le secteur bancaire."
texte_D = "L'utilisation de modèles linguistiques avancés permet d'améliorer l'automatisation des tâches répétitives dans le secteur bancaire."

decision_2, score_2 = predire_plagiat(texte_C, texte_D)
print(f"--- CAS 2 : Plagiat ---")
print(f"Score de Similarité: {score_2:.4f} (Seuil: 0.60)")
print(f"Décision: {decision_2}")
print("-" * 30)

# Vous pouvez maintenant remplacer texte_A, texte_B, texte_C, texte_D par les vrais contenus de vos mémoires.

--- CAS 1 : Non-Plagiat ---
Score de Similarité: 0.0000 (Seuil: 0.60)
Décision: NON-PLAGIAT (0)
------------------------------
--- CAS 2 : Plagiat ---
Score de Similarité: 1.0000 (Seuil: 0.60)
Décision: PLAGIAT DÉTECTÉ (1)
------------------------------


In [ ]:
# Définition des variables
COL_MEMOIRE = 'texte_memoire'
COL_SOURCE = 'texte_source'
COL_SCORE_SIMILARITE = 'X_similarite'
COL_CIBLE = 'y_plagiat_cree' # Colonne de décision binaire (0 ou 1)
SEUIL = 0.60

# AFFICHAGE DES RÉSULTATS PAR DÉCISION

print(f"--- ANALYSE DÉTAILLÉE DES CAS (Seuil de Plagiat : {SEUIL}) ---")
print("-" * 50)

# filtre
df_plagiat = df[df[COL_CIBLE] == 1]

print(f"\n CAS DE PLAGIAT DÉTECTÉ (Score >= {SEUIL}) : {len(df_plagiat)} cas")
print("   (La 'Mémoire' est considérée comme très similaire à la 'Source'.)")
print("-" * 50)

if not df_plagiat.empty:
    # Affiche les colonnes pertinentes pour ces cas
    print(df_plagiat[[COL_MEMOIRE, COL_SOURCE, COL_SCORE_SIMILARITE, COL_CIBLE]].head(10))
else:
    print("Aucun cas de plagiat détecté selon ce seuil.")


# Filtrage des Cas de NON-PLAGIAT (y = 0)
df_non_plagiat = df[df[COL_CIBLE] == 0]

print(f"\n CAS DE NON-PLAGIAT (Score < {SEUIL}) : {len(df_non_plagiat)} cas")
print("   (La 'Mémoire' est considérée comme suffisamment différente de la 'Source'.)")
print("-" * 50)

if not df_non_plagiat.empty:
    # Affiche les colonnes pertinentes pour ces cas
    print(df_non_plagiat[[COL_MEMOIRE, COL_SOURCE, COL_SCORE_SIMILARITE, COL_CIBLE]].head(10))
else:
    print("Toutes les paires ont dépassé le seuil de plagiat.")


# --- 3. Synthèse ---
print("-" * 50)
print(f"Synthèse : {len(df_plagiat)} cas de Plagiat (1) et {len(df_non_plagiat)} cas de Non-Plagiat (0).")

--- ANALYSE DÉTAILLÉE DES CAS (Seuil de Plagiat : 0.6) ---
--------------------------------------------------

 CAS DE PLAGIAT DÉTECTÉ (Score >= 0.6) : 467 cas
   (La 'Mémoire' est considérée comme très similaire à la 'Source'.)
--------------------------------------------------
                                        texte_memoire  \
1   L’impact des réseaux sociaux sur la réussite s...   
9    Conservation des espèces menacées dans les parcs   
11  Les effets du changement climatique sur la bio...   
17  Conception d’une plateforme web de dépôt de mé...   
19    Développement d’une application mobile de suivi   
23   Conservation des espèces menacées dans les parcs   
24  L’influence de la digitalisation sur la perfor...   
25  Les effets du changement climatique sur la bio...   
29  Sécurité et cryptographie dans le cloud computing   
43  Sécurité et cryptographie dans le cloud computing   

                                         texte_source  X_similarite  \
1       Influence des